In [ ]:
bv = True # batched vecchia or gpgp
gpgp = False # batched vecchia or gpgp
dep = True # vecchia or independent block for bv
grouping = True # use grouping in gpgp or not
knn = True # use knn in gpgp or not
morton = True # ordering 
minmax = False # ordering
exact = False # if the exageostat results is available

# 1. Prepare the folders

In [ ]:
import os

def fold_create(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created successfully.")
    else:
        print(f"Folder '{folder_name}' already exists.")

In [ ]:
import nbformat
import shutil
import subprocess

def modify_notebook(filename, code_block_index, new_code):
    # Load the Jupyter Notebook file
    with open(filename, 'r') as f:
        notebook = nbformat.read(f, as_version=nbformat.NO_CONVERT)

    # Access the code block and modify its content
    notebook.cells[code_block_index].source = new_code

    # Save the modified notebook to a new file
    new_filename = 'modified_notebook.ipynb'
    with open(new_filename, 'w') as f:
        nbformat.write(notebook, f)

    # Replace the old file with the new file
    shutil.move(new_filename, filename)
    # Run the modified notebook
    subprocess.run(['jupyter', 'nbconvert', '--execute', '--inplace', '--to', 'notebook', filename])



### BV or GpGp

In [ ]:
range= [0.017526, 0.021080, 0.030933, 0.052579, 0.063240, 0.092798, 0.140211, 0.168639, 0.247462]
alpha= [2.3, 1.5, 0.6, 2.3, 1.5, 0.6, 2.3, 1.5, 0.6]


for i in range(len(alpha)):
    ra = range[i]
    al = alpha[i]
    if exact:
        os.system(f"cp ./output_exact.csv ./data_40k_{ra}_{al}_bv/output_exact.csv")
    # train helper file copy and modify 
    if bv: 
        # data and fold create
        fold_create(f"data_40k_{ra}_{al}_bv")
        os.system(f"cp -r ./synData/synthetic_ds_{ra}_{al} ./data_40k_{ra}_{al}_bv/synthetic_ds")
        if dep:
            os.system(f"cp ./train_helper_40k_base.ipynb ./data_40k_{ra}_{al}_bv/train_helper_40k_{ra}_{al}_bv_dependent.ipynb")
            ## Specify the filename, code block index, and new code & Call the function to modify the notebook
            jupyter_path = f"./data_40k_{ra}_{al}_bv/train_helper_40k_{ra}_{al}_bv_dependent.ipynb"
            new_code = f"params = [1.5, {ra}, {al}]\n" + \
                        "bv = True\n" + \
                        "gpgp = False\n" + \
                        "dep = True\n" + \
                        "grouping = True\n" + \
                        "knn = True\n" + \
                        "morton = True\n" + \
                        "minmax = False\n" + \
                        "exact = False"
            modify_notebook(jupyter_path, 2, new_code)
        else:
            os.system(f"cp ./train_helper_40k_base.ipynb ./data_40k_{ra}_{al}_bv/train_helper_40k_{ra}_{al}_bv_independent.ipynb")
            ## Specify the filename, code block index, and new code & Call the function to modify the notebook
            jupyter_path = f"./data_40k_{ra}_{al}_bv/train_helper_40k_{ra}_{al}_bv_independent.ipynb"
            new_code = f"params = [1.5, {ra}, {al}]\n" + \
                        "bv = True\n" + \
                        "gpgp = False\n" + \
                        "dep = False\n" + \
                        "grouping = True\n" + \
                        "knn = True\n" + \
                        "morton = True\n" + \
                        "minmax = False\n" + \
                        "exact = False" 
            modify_notebook(jupyter_path, 2, new_code)
    else:
        fold_create(f"data_40k_{ra}_{al}_gpgp")
        os.system(f"cp -r ./synData/synthetic_ds_{ra}_{al} ./data_40k_{ra}_{al}_gpgp/synthetic_ds")
        os.system(f"cp ./train_helper_40k_base.R ./data_40k_{ra}_{al}_gpgp/train_helper_40k_base.R")
        if grouping:
            if knn:
                if morton:
                    os.system(f"cp ./train_helper_40k_base.ipynb ./data_40k_{ra}_{al}_gpgp/train_helper_40k_{ra}_{al}_gpgp_knn_grouping.ipynb")
                    ## Specify the filename, code block index, and new code & Call the function to modify the notebook
                    jupyter_path = f"./data_40k_{ra}_{al}_gpgp/train_helper_40k_{ra}_{al}_gpgp_knn_grouping.ipynb"
                    new_code = f"params = [1.5, {ra}, {al}]\n" + \
                            "bv = False\n" + \
                            "gpgp = True\n" + \
                            "dep = True\n" + \
                            "grouping = True\n" + \
                            "knn = True\n" + \
                            "morton = True\n" + \
                            "minmax = False\n"  + \
                            "exact = False"
                    modify_notebook(jupyter_path, 2, new_code)
                if minmax:
                    os.system(f"cp ./train_helper_40k_base.ipynb ./data_40k_{ra}_{al}_gpgp/train_helper_40k_{ra}_{al}_gpgp_knn_grouping.ipynb")
                    ## Specify the filename, code block index, and new code & Call the function to modify the notebook
                    jupyter_path = f"./data_40k_{ra}_{al}_gpgp/train_helper_40k_{ra}_{al}_gpgp_knn_grouping.ipynb"
                    new_code = f"params = [1.5, {ra}, {al}]\n" + \
                            "bv = False\n" + \
                            "gpgp = True\n" + \
                            "dep = True\n" + \
                            "grouping = True\n" + \
                            "knn = True\n" + \
                            "morton = False\n" + \
                            "minmax = True\n"  + \
                            "exact = False"
                    modify_notebook(jupyter_path, 2, new_code)
            else:
                pass #TBD

### 2. Bash create

- bv dependent morton

In [ ]:
content_bv_dependent_morton = '''#!/bin/bash

# Define an array with the different values
range=(0.017526 0.021080 0.030933 0.052579 0.063240 0.092798 0.140211 0.168639 0.247462)
alpha=(2.3 1.5 0.6 2.3 1.5 0.6 2.3 1.5 0.6)

# Loop over the array and execute the commands with each value
for index in {0..8}
do
    mv "./data_40k_${range[index]}_${alpha[index]}_bv" "../data"
    cd ..
    bash ./data/run_script.sh
    mv "./data" "./dat1_bv_dependent_morton/data_40k_${range[index]}_${alpha[index]}_bv"
    cd ./dat1_bv_dependent_morton
done
'''

- bv independent morton

In [ ]:
content_bv_independent_morton = '''#!/bin/bash

# Define an array with the different values
range=(0.017526 0.021080 0.030933 0.052579 0.063240 0.092798 0.140211 0.168639 0.247462)
alpha=(2.3 1.5 0.6 2.3 1.5 0.6 2.3 1.5 0.6)

# Loop over the array and execute the commands with each value
for index in {0..8}
do
    mv "./data_40k_${range[index]}_${alpha[index]}_bv" "../data"
    cd ..
    bash ./data/run_script.sh
    mv "./data" "./dat1_bv_independent_morton/data_40k_${range[index]}_${alpha[index]}_bv"
    cd ./dat1_bv_independent_morton
done
'''

- gpgp group + knn + morton/minmax

In [ ]:
content_gpgp_knn_group = '''#!/bin/bash

# Define an array with the different values
range=(0.017526 0.021080 0.030933 0.052579 0.063240 0.092798 0.140211 0.168639 0.247462)
alpha=(2.3 1.5 0.6 2.3 1.5 0.6 2.3 1.5 0.6)

# Loop over the array and execute the commands with each value
for index in {0..8}
do  
    echo "[INFO] ./data_40k_${range[index]}_${alpha[index]}_gpgp is starting now!"
    cd "./data_40k_${range[index]}_${alpha[index]}_gpgp"
    Rscript ./train_helper_40k_base.R
    cd ..
    echo "[INFO] ./data_40k_${range[index]}_${alpha[index]}_gpgp is done!"
done
'''

 - write in

In [ ]:
# Open the file in write mode using a context manager
if bv:
    with open('bash_bv_base.sh', 'w') as file:
        if dep:
            file.write(content_bv_dependent_morton)
        else:
            file.write(content_bv_independent_morton)
if gpgp:
    if grouping:
        if knn:
            with open('bash_gpgp_base.sh', 'w') as file:
                file.write(content_gpgp_knn_group)


### run after training your model

In [ ]:
# import nbformat
# import shutil
# import subprocess

# def run_notebook(filename):
#     # Run the modified notebook
#     subprocess.run(['jupyter', 'nbconvert', '--execute', '--inplace', '--to', 'notebook', filename])

# range = [0.03, 0.1, 0.3]
# alpha = [0.5, 1.5]

# for ra in range:
#     for al in alpha:
#         # train helper file copy and modify 
#         if bv: 
#             if dep:
#                 jupyter_path = f"./data_40k_{ra}_{al}_bv/train_helper_40k_{ra}_{al}_bv_dependent.ipynb"
#             else:
#                 jupyter_path = f"./data_40k_{ra}_{al}_bv/train_helper_40k_{ra}_{al}_bv_independent.ipynb"
#         else:
#             if grouping:
#                 jupyter_path = f"./data_40k_{ra}_{al}_gpgp/train_helper_40k_{ra}_{al}_gpgp_knn_grouping.ipynb"
#             elif knn:
#                 jupyter_path = f"./data_40k_{ra}_{al}_gpgp/train_helper_40k_{ra}_{al}_gpgp_knn.ipynb"
#             else:
#                 jupyter_path = f"./data_40k_{ra}_{al}_gpgp/train_helper_40k_{ra}_{al}_gpgp.ipynb"
#         run_notebook(jupyter_path)

## Y-axis Unified

This block is used to unify the y-axis for all figures. Note: you have to modify the binary value and the folder name inside the code, such as 
- `bv=Ture`, 
- `dat1_gpgp_knn_group_morton`

In [ ]:
# def run_notebook(filename):
#     # Run the modified notebook
#     subprocess.run(['jupyter', 'nbconvert', '--execute', '--inplace', '--to', 'notebook', filename])


# range = [0.03, 0.1, 0.3]
# alpha = [0.5, 1.5]

# yaxis_info = []
# # (0.03, 0.5)
# yaxis_info.append("time_log=False\n" + 
#                   "iter_log=False\niter_lim=None\n" +
#                   "vari_lim=None\n" + 
#                   "range_lim=[0.01, 0.10]\n" + 
#                   "smooth_lim=[0.15, 1.50]\n" +
#                   "com_log=True\n" +
#                   "com_lim=None")
# # (0.03, 1.5)
# yaxis_info.append("time_log=False\n" + 
#                   "iter_log=False\niter_lim=None\n" +
#                   "vari_lim=None\n" + 
#                   "range_lim=[0.018, 0.06]\n" + 
#                   "smooth_lim=[0.6, 3.2]\n" +
#                   "com_log=True\n" +
#                   "com_lim=None")

# # (0.1, 0.5)
# yaxis_info.append("time_log=False\n" + 
#                   "iter_log=False\niter_lim=None\n" +
#                   "vari_lim=None\n" + 
#                   "range_lim=[0.02, 0.30]\n" + 
#                   "smooth_lim=[0.3, 1.0]\n" +
#                   "com_log=False\n" +
#                   "com_lim=None")

# # (0.1, 1.5)
# yaxis_info.append("time_log=False\n" + 
#                   "iter_log=False\niter_lim=None\n" +
#                   "vari_lim=[0., 2.0]\n" + 
#                   "range_lim=[0.02, 0.32]\n" + 
#                   "smooth_lim=[0.7, 3.5]\n" +
#                   "com_log=False\n" +
#                   "com_lim=None")

# # (0.3, 0.5)
# yaxis_info.append("time_log=False\n" + 
#                   "iter_log=False\niter_lim=None\n" +
#                   "vari_lim=None\n" + 
#                   "range_lim=[0.02, 1.2]\n" + 
#                   "smooth_lim=[0.3, 1.0]\n" +
#                   "com_log=False\n" +
#                   "com_lim=None")

# # (0.3, 1.5)
# yaxis_info.append("time_log=False\n" + 
#                   "iter_log=False\niter_lim=None\n" +
#                   "vari_lim=[0., 3.0]\n" + 
#                   "range_lim=[0.0, 1.0]\n" + 
#                   "smooth_lim=[0.8, 4.2]\n" +
#                   "com_log=False\n" +
#                   "com_lim=None")

# index_params=0
# for ra in range:
#     for al in alpha:
#         # used for change parameters
#         new_code = yaxis_info[index_params]
        
#         new_code1 = f"params = [1.0, {ra}, {al}]\nbv = True\ngpgp = False\ndep = True\nknn = False\ngrouping = False\nmorton = True\nminmax = False"        
#         new_code2 = f"params = [1.0, {ra}, {al}]\nbv = True\ngpgp = False\ndep = False\nknn = False\ngrouping = False\nmorton = True\nminmax = False"        
#         new_code3 = f"params = [1.0, {ra}, {al}]\nbv = False\ngpgp = True\ndep = True\nknn = True\ngrouping = True\nmorton = False\nminmax = True"        
#         new_code4 = f"params = [1.0, {ra}, {al}]\nbv = False\ngpgp = True\ndep = True\nknn = True\ngrouping = True\nmorton = True\nminmax = False"        

#         index_params +=1
#         # train helper file copy and modify 
#         # bv : dependent 
#         os.system(f"cp ./train_helper_40k_base.ipynb ../dat1_bv_dependent_morton/data_40k_{ra}_{al}_bv/train_helper_40k_{ra}_{al}_bv_dependent_morton.ipynb")
#         ## Specify the filename, code block index, and new code & Call the function to modify the notebook
#         jupyter_path = f"../dat1_bv_dependent_morton/data_40k_{ra}_{al}_bv/train_helper_40k_{ra}_{al}_bv_dependent_morton.ipynb"
#         modify_notebook(jupyter_path, 3, new_code)
#         modify_notebook(jupyter_path, 2, new_code1)
#         os.system(f"rm -rf ../dat1_bv_dependent_morton/data_40k_{ra}_{al}_bv/fig_40k")
#         run_notebook(jupyter_path)
#         # bv : independent 
#         os.system(f"cp ./train_helper_40k_base.ipynb ../dat1_bv_independent_morton/data_40k_{ra}_{al}_bv/train_helper_40k_{ra}_{al}_bv_independent_morton.ipynb")
#         ## Specify the filename, code block index, and new code & Call the function to modify the notebook
#         jupyter_path = f"../dat1_bv_independent_morton/data_40k_{ra}_{al}_bv/train_helper_40k_{ra}_{al}_bv_independent_morton.ipynb"
#         modify_notebook(jupyter_path, 3, new_code)
#         modify_notebook(jupyter_path, 2, new_code2)
#         os.system(f"rm -rf ../dat1_bv_independent_morton/data_40k_{ra}_{al}_bv/fig_40k")
#         run_notebook(jupyter_path)
        
#         # gpgp : group + knn + minimmax
#         os.system(f"cp ./train_helper_40k_base.ipynb ../dat1_gpgp_knn_group_minmiax/data_40k_{ra}_{al}_gpgp/train_helper_40k_{ra}_{al}_gpgp_knn_grouping_minimax.ipynb")
#         ## Specify the filename, code block index, and new code & Call the function to modify the notebook
#         jupyter_path = f"../dat1_gpgp_knn_group_minmiax/data_40k_{ra}_{al}_gpgp/train_helper_40k_{ra}_{al}_gpgp_knn_grouping_minimax.ipynb"
#         modify_notebook(jupyter_path, 3, new_code)
#         modify_notebook(jupyter_path, 2, new_code3)
#         os.system(f"rm -rf ../dat1_gpgp_knn_group_minmiax/data_40k_{ra}_{al}_gpgp/fig_40k")
#         run_notebook(jupyter_path)
        
#         # gpgp : group + knn + morton
#         os.system(f"cp ./train_helper_40k_base.ipynb ../dat1_gpgp_knn_group_morton/data_40k_{ra}_{al}_gpgp/train_helper_40k_{ra}_{al}_gpgp_knn_grouping_morton.ipynb")
#         ## Specify the filename, code block index, and new code & Call the function to modify the notebook
#         jupyter_path = f"../dat1_gpgp_knn_group_morton/data_40k_{ra}_{al}_gpgp/train_helper_40k_{ra}_{al}_gpgp_knn_grouping_morton.ipynb"
#         modify_notebook(jupyter_path, 3, new_code)
#         modify_notebook(jupyter_path, 2, new_code4)
#         os.system(f"rm -rf ../dat1_gpgp_knn_group_morton/data_40k_{ra}_{al}_gpgp/fig_40k")
#         run_notebook(jupyter_path)